In [6]:
from bs4 import BeautifulSoup
import requests as req
import re

# 네이버 웹툰 URL
res = req.get('https://comic.naver.com/webtoon/weekday.nhn')
soup = BeautifulSoup(res.text)

# {웹툰 이름 : 웹툰 번호} 딕셔너리
webtoon_dict = {}
for i in soup.select('.title'):
    title = i.get_text(strip=True)
    titleId = re.findall(r".*titleId=(\d+)&", i['href']).pop()
    webtoon_dict[title] = titleId
    
def get_soup(titleId):
    """ Returns soup of response that is user_in webtoon page
    :param titleId: (string) webtoon titleId number
    :return: (object) parsed page of response
    """
    res = req.get(f'https://comic.naver.com/webtoon/list.nhn?titleId={webtoon_dict[titleId]}')
    soup = BeautifulSoup(res.text)
    return soup

def webtoon_info_detail(user_in):
    """ Prints detail info of user_in webtoon
    :param user_in: user input webtoon title
    :return: None
    """
    print("\n", f"# 정보 ({user_in})")
    soup = get_soup(user_in)
    author = soup.select_one('.detail .wrt_nm').get_text(strip=True)
    desc = soup.select_one('.detail p').get_text('\n', strip=True)
    genre = soup.select_one('.detail .genre').get_text(strip=True)
    # genre_list = [i.strip() for i in soup.select_one('.detail .genre').get_text(strip=True).split(',')]
    print(f"작가: {author}\n"
          f"설명: {desc}\n"
          f"장르: {genre}\n")
    
def episode_list(user_in):
    """ Prints title, rating, date of user_in webtoon
    :param user_in: user input webtoon title
    :return: None
    """
    print("\n", f"# 에피소드목록 ({user_in})")
    soup = get_soup(user_in)
    title = soup.select('.title a')
    rating = soup.select('.rating_type strong')
    date = soup.select('.num')
    data = [dict(title=x.get_text(), rating=y.get_text(), date=z.get_text()) for x, y, z in zip(title, rating, date)]
    for d in data:
        print("{}\n  평점: {}, 날짜: {}".format(d['title'], d['rating'], d['date']))
        
def search_webtoon(user_in, choice):
    """ Search user input webtoon and call function
        This is sort of middleware
    :param user_in: user input webtoon title
    :param choice: user want what to get
    :return: None
    """
    find_data = [i for i in webtoon_dict if user_in in i]
    if choice == '1':
        print("\n", "# 정보를 가져올 웹툰선택")
    elif choice == '2':
        print("\n", "# 에피소드 목록을 가져올 웹툰선택")
    for n, title in enumerate(find_data):
        print(f'{n + 1}. {title}')
    print("0. 이전")
    webtoon_choice = int(input("> 웹툰 선택: "))
    if not webtoon_choice:
        return
    elif choice == '1':
        webtoon_info_detail(find_data[webtoon_choice - 1])
    elif choice == '2':
        episode_list(find_data[webtoon_choice - 1])
        
# 웹툰 크롤러 반복 실행
while True:
    print("## 웹툰 크롤러 ##")
    print("1. 정보 \n"
          "2. 에피소드 목록 \n"
          "0. 나가기 \n")
    choice = input("> 입력: ")
    if choice == '0':
        print("Thank you")
        break
    elif choice == '1' or '2':
        user_in = input("> 웹툰명을 입력하세요: ")
        search_webtoon(user_in, choice)
    else:
        print("!!! 숫자로 입력해 주세요 !!!")


## 웹툰 크롤러 ##
1. 정보 
2. 에피소드 목록 
0. 나가기 

> 입력: 1
> 웹툰명을 입력하세요: 신의

 # 정보를 가져올 웹툰선택
1. 신의 탑
2. 신의 언어
0. 이전
> 웹툰 선택: 2

 # 정보 (신의 언어)
작가: 장래혁
설명: 퇴마계의 신개념 영웅!
프로그래머 난새의 마계 전투가 시작된다!
장르: 스토리, 판타지

## 웹툰 크롤러 ##
1. 정보 
2. 에피소드 목록 
0. 나가기 

> 입력: 2
> 웹툰명을 입력하세요: 신의

 # 에피소드 목록을 가져올 웹툰선택
1. 신의 탑
2. 신의 언어
0. 이전
> 웹툰 선택: 2

 # 에피소드목록 (신의 언어)
2부 96화 동장군 (3)
  평점: 9.48, 날짜: 2019.12.02
2부 95화 동장군 (2)
  평점: 9.20, 날짜: 2019.11.25
2부 94화 동장군 (1)
  평점: 8.70, 날짜: 2019.11.18
2부 93화 여우족 우두머리 (3)
  평점: 9.23, 날짜: 2019.11.11
2부 92화 여우족 우두머리 (2)
  평점: 8.60, 날짜: 2019.11.04
2부 91화 여우족 우두머리 (1)
  평점: 8.42, 날짜: 2019.10.28
2부 90화 블랙아웃 (3)
  평점: 6.74, 날짜: 2019.10.21
2부 89화 블랙아웃 (2)
  평점: 6.88, 날짜: 2019.10.14
2부 88화 블랙아웃 (1)
  평점: 5.22, 날짜: 2019.10.07
2부 87화 그녀의 비밀병기 (3)
  평점: 6.45, 날짜: 2019.09.30
## 웹툰 크롤러 ##
1. 정보 
2. 에피소드 목록 
0. 나가기 

> 입력: 0
Thank you
